# 🧠 Claude Memory on Amazon Bedrock
### Build Your First AI with Perfect Memory in 60 Minutes!

<div style="background: #f0f9ff; padding: 20px; border-radius: 8px; border-left: 4px solid #0369a1;">
<strong>What You'll Build:</strong><br>
An AI assistant that remembers everything about you - permanently across different conversations!
</div>

## 📚 Learning Outcomes

By the end of this tutorial, you will:
- ✅ Understand how Claude Sonnet 4.5's memory system works
- ✅ Implement a minimal client for memory operations
- ✅ Store information that persists across conversations
- ✅ Build personalized AI interactions based on memories
- ✅ Feel confident deploying memory-enabled applications

## ⏱️ Time Required
**45 minutes total** (setup and hands-on practice)

## 📋 Tutorial Structure

### Part 1: Connection Test (5 min)
Verify your connection to Claude Sonnet 4.5

### Part 2: Storing Your First Memory (10 min)
Learn how Claude stores information autonomously

### Part 3: Retrieving Memories in New Sessions (10 min)
See memory persistence across independent conversations

### Part 4: Updating and Managing Memories (10 min)
Dynamically update stored information

### Part 5: Building a Personalized Assistant (10 min)
Create context-aware, personalized interactions

---

## 🎯 Before You Begin

### Prerequisites

This tutorial assumes you have:
- ✅ AWS account with Bedrock access enabled
- ✅ Claude Sonnet 4.5 model access approved
- ✅ AWS credentials configured (`aws configure` completed)
- ✅ Python 3.8+ installed
- ✅ boto3 package installed (`pip install boto3`)

**Need setup help?** See the [Quick Start Guide](docs/tutorials/claude-memory-quickstart.md) for detailed AWS setup instructions.

---
# Part 1: Setup 🚀

In this section, we'll set up everything needed to work with Claude's memory system.

## Setup: Minimal Client Code

The following cells contain everything you need to work with Claude's memory system. We'll build it step-by-step:

1. **Imports and AWS Connection** - Connect to Claude via AWS Bedrock
2. **Memory Tool Handler** - Handle Claude's requests to store/read/update memories  
3. **Chat Function** - Manage conversations with automatic tool use loops

In [ ]:
import boto3
import json
from datetime import datetime
from pathlib import Path
from botocore.config import Config

# AWS Bedrock configuration
config = Config(
   retries = {
      'total_max_attempts': 1000,
      'mode': 'standard'
   }
)

# Model configuration
MODEL_ID = "global.anthropic.claude-sonnet-4-5-20250929-v1:0"

# AWS session setup
session = boto3.session.Session()
bedrock_rt = session.client(
    service_name='bedrock-runtime',
    config=config
)

## Building the Memory Tool Handler

Now that we have our basic connection set up, we need to create a function that handles Claude's memory tool requests.

### What is the Memory Tool?

Claude Sonnet 4.5 has built-in memory capabilities - it can autonomously decide to **store**, **read**, and **update** information about you. When Claude wants to use memory, it sends tool requests to your application with commands like:
- `view` - Read existing memories
- `create` - Store new memories
- `str_replace` - Update existing memories

### Why Do We Need a Handler?

Claude doesn't directly access your file system - instead, it makes tool requests that we need to implement. Think of it like this:
- **Claude (the brain)**: "I want to remember that this user loves Python"
- **Our handler (the hands)**: Writes that information to a `.md` file in the `./memories/` folder

### How It Works

Our `handle_memory_tool()` function translates Claude's requests into file operations:
1. **List memories**: When Claude asks to `view /memories`, we return all `.md` files
2. **Read a memory**: When Claude asks for a specific memory file (e.g., `/memories/user_profile.md`), we return its contents
3. **Create memory**: When Claude wants to remember something new, we create a new `.md` file
4. **Update memory**: When Claude wants to update existing information, we modify the file

**Note**: Claude will use `.md` file extensions directly thanks to a system prompt we'll configure later in the chat function. This means our handler can use paths exactly as Claude provides them - no conversion needed!

Let's implement this handler:

In [ ]:
# Create memories directory for file-based storage
MEMORY_DIR = Path("./memories")
MEMORY_DIR.mkdir(exist_ok=True)

print("✅ AWS Bedrock connection ready")
print(f"✅ Memory directory: {MEMORY_DIR.absolute()}")

def handle_memory_tool(tool_input):
    """File-based handler for Claude Sonnet 4.5's memory tool requests"""
    command = tool_input.get('command')
    path = tool_input.get('path', '')
    
    if command == 'view':
        if path == '/memories':
            # List all memory files
            memories = []
            for md_file in MEMORY_DIR.glob('*.md'):
                memories.append({
                    'path': f'/memories/{md_file.name}',
                    'content': md_file.read_text(),
                    'created': datetime.fromtimestamp(md_file.stat().st_ctime).isoformat()
                })
            return {'memories': memories}
        elif path.startswith('/memories/'):
            # Get specific memory file
            filename = path.split('/')[-1]
            filepath = MEMORY_DIR / filename
            if filepath.exists():
                return {
                    'memory': {
                        'path': path,
                        'content': filepath.read_text(),
                        'created': datetime.fromtimestamp(filepath.stat().st_ctime).isoformat()
                    }
                }
            return {'memory': {'error': 'not found'}}
    
    elif command == 'create':
        # Create new memory file
        filename = path.split('/')[-1] if '/' in path else f"mem_{len(list(MEMORY_DIR.glob('*.md')))}.md"
        filepath = MEMORY_DIR / filename
        filepath.write_text(tool_input.get('file_text', ''))
        print(f"   💾 Stored memory file: {filename}")
        return {'success': True, 'created': filename}
    
    elif command == 'str_replace':
        # Update existing memory file
        filename = path.split('/')[-1]
        filepath = MEMORY_DIR / filename
        if filepath.exists():
            content = filepath.read_text()
            old_str = tool_input.get('old_str', '')
            new_str = tool_input.get('new_str', '')
            updated_content = content.replace(old_str, new_str)
            filepath.write_text(updated_content)
            print(f"   ✏️ Updated memory file: {filename}")
            return {'success': True}
        return {'error': 'Memory not found'}
    
    return {'status': 'handled', 'command': command}

print("✅ Memory handler function ready")

## Creating the Chat Function with Tool Support

Now we have a handler for memory operations, but we need a way to actually **talk to Claude** and handle its tool requests automatically.

### Why Not Just Call the API Directly?

Claude's memory system uses a **tool use loop**. Here's what happens:
1. You send a message to Claude
2. Claude might respond with text OR with a tool use request (to access memory)
3. If it requests a tool, you call `handle_memory_tool()` and send the result back
4. Claude might then request MORE tools, or finally respond with text
5. Repeat until Claude gives a final text response

This back-and-forth needs to happen automatically - you don't want to manually check for tool requests each time.

### What Does chat_with_claude() Do?

Our `chat_with_claude()` function handles this entire loop:
- **Manages conversation history**: Keeps track of all messages (yours and Claude's)
- **Detects tool requests**: Automatically checks if Claude wants to use the memory tool
- **Calls our handler**: Routes tool requests to `handle_memory_tool()`
- **Loops until complete**: Keeps going until Claude gives a final text response

### System Prompt Configuration

To ensure Claude uses `.md` file extensions (instead of defaulting to `.txt`), we'll configure a system prompt:

```python
SYSTEM_PROMPT = "When using the memory tool, always use .md file extensions for markdown-formatted memories."
```

**Why use `.md` extensions?**
- Markdown files render nicely in editors and GitHub
- Supports formatting like **bold**, *italics*, lists, code blocks
- Better readability for human review
- No path conversion logic needed in our handler

This system prompt will be included in every API call to Claude, ensuring consistent `.md` usage throughout.

### The Tool Use Loop in Action

```
You: "Remember that I love Python"
  → Claude: [tool_use: create memory about Python preference]
    → Handler: Creates ./memories/user_preferences.md
    → [tool_result: success]
  → Claude: "I've stored that you love Python. I'll remember this!"
```

Let's implement this chat function with automatic tool handling:

In [ ]:
# System prompt to ensure Claude uses .md extensions for memories
SYSTEM_PROMPT = "When using the memory tool, always use .md file extensions for markdown-formatted memories."

def chat_with_claude(message, conversation_history=None):
    """Send a message to Claude Sonnet 4.5 and handle any tool use"""
    messages = conversation_history or []
    
    # Add user message
    messages.append({
        "role": "user",
        "content": [{"type": "text", "text": message}]
    })
    
    print(f"\n👤 You: {message}\n")
    
    # Keep handling tool uses until Claude Sonnet 4.5 stops requesting them
    max_iterations = 5  # Safety limit
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        
        # Prepare request body
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "anthropic_beta": ["context-management-2025-06-27"],
            "system": [{"type": "text", "text": SYSTEM_PROMPT}],
            "tools": [{
                "type": "memory_20250818",
                "name": "memory"
            }],
            "max_tokens": 4000,
            "messages": messages,
        }
        
        # Send to Claude Sonnet 4.5
        response = bedrock_rt.invoke_model(
            modelId=MODEL_ID,
            body=json.dumps(body)
        )
        result = json.loads(response["body"].read())
        
        # Track if we need to handle tool use
        has_tool_use = False
        tool_uses = []
        
        # Process response
        for content in result.get('content', []):
            if content['type'] == 'text':
                print(f"🤖 Claude: {content['text']}\n")
            
            elif content['type'] == 'tool_use':
                has_tool_use = True
                tool_uses.append(content)
                print(f"🔧 Claude uses memory tool: {content['input'].get('command', 'unknown')}")
                print(f"   Path: {content['input'].get('path', 'N/A')}")
                if content['input'].get('file_text'):
                    print(f"   Creating memory file...\n")
        
        # Add Claude's response to conversation
        messages.append({
            "role": "assistant",
            "content": result['content']
        })
        
        # If no tool use, we're done
        if not has_tool_use:
            break
        
        # Handle all tool uses
        tool_results = []
        for tool_use in tool_uses:
            tool_result = handle_memory_tool(tool_use['input'])
            tool_results.append({
                "type": "tool_result",
                "tool_use_id": tool_use['id'],
                "content": json.dumps(tool_result)
            })
        
        # Add tool results to conversation
        messages.append({
            "role": "user",
            "content": tool_results
        })
        
        # Continue loop to get Claude's next response
    
    return messages

print("✅ Chat function ready")

### 🎯 Checkpoint: Setup Complete!

**What you just did:**
- ✅ Imported required libraries
- ✅ Connected to AWS Bedrock
- ✅ Created memory directory for file-based storage
- ✅ Built memory tool handler to translate Claude's requests into file operations
- ✅ Built chat function with automatic tool use loop

**Progress**: [█░░░░] 20% complete

### What You've Built: The Complete Architecture

```
┌──────────────────────────────────────────────────────────────┐
│                     Your Application                         │
│                                                              │
│  ┌──────────────────┐    ┌──────────────────┐                │
│  │  chat_with_      │───▶│  handle_memory_  │                │
│  │  claude()        │◀───│  tool()          │                │
│  │                  │    │                  │                │
│  │ - Sends messages │    │ - view memories  │                │
│  │ - Handles tool   │    │ - create files   │    ┌─────────┐ │
│  │   use loop       │    │ - update files   │───▶│memories/│ │
│  │ - System prompt  │    │                  │    │ *.md    │ │
│  └──────────────────┘    └──────────────────┘    └─────────┘ │
│           │                                                  │
│           ▼                                                  │
│  ┌──────────────────┐                                        │
│  │  AWS Bedrock     │                                        │
│  │  Claude Sonnet   │                                        │
│  │  4.5             │                                        │
│  └──────────────────┘                                        │
└──────────────────────────────────────────────────────────────┘
```

**Key Insight**: Claude autonomously decides when to use memory. You just provide the tools!

You're ready to store your first memory!

---
# Part 2: Storing Your First Memory 📝

## What You'll Learn
How Claude's memory system works and how to store information autonomously.

## Your Goal
Store information about yourself that Claude can recall later.

## 💡 Key Concept: The Memory System

Think of it like this:
- 📝 **Claude** decides what to write in a notebook
- 📂 **Your code** provides the notebook and retrieves pages when asked
- 🔄 **Memories** persist as `.md` files on your computer

You don't tell Claude what to remember - it figures that out automatically!

In [ ]:
print("=" * 60)
print("SESSION 1: Teaching Claude About You")
print("=" * 60)

# Tell Claude about yourself
conversation1 = chat_with_claude(
    "Hi Claude! Please remember these things about me: "
    "I'm a Python developer who loves machine learning, "
    "I use VSCode with dark theme, "
    "and my favorite libraries are numpy, pandas, and scikit-learn."
)

### 💡 Pro Tip: Viewing Memory Files

Since these are real `.md` files on your disk, you can:
- Run `ls memories/` to list them
- Open them in VSCode/Jupyter Lab/any text editor
- Read them with: `cat memories/user_profile.md`
- Edit them manually if needed

Let's view what was created!

In [ ]:
print("📦 Memory Files Created:")
print("=" * 60)

memory_files = list(MEMORY_DIR.glob('*.md'))
print(f"Total memory files: {len(memory_files)}")
print(f"Memory directory: {MEMORY_DIR.absolute()}\n")

for md_file in sorted(memory_files):
    print(f"File: {md_file.name}")
    print(f"Path: {md_file.absolute()}")
    print(f"Size: {md_file.stat().st_size} bytes")
    print(f"Created: {datetime.fromtimestamp(md_file.stat().st_ctime).isoformat()}")
    
    content = md_file.read_text()
    if content:
        preview = content[:200] + "..." if len(content) > 200 else content
        print(f"\nContent preview:\n{preview}")
    print("-" * 60)

if len(memory_files) > 0:
    print("\n🎉 SUCCESS! Claude stored its first memory as a .md file!")
    print(f"💡 You can open these files in any text editor!")
else:
    print("\n⚠️ No memory files created. Try running the previous cell again.")

## Session 2: Test Memory Recall

Let's start a completely fresh conversation with no history.

In [ ]:
print("=" * 60)
print("SESSION 2: New Conversation - Testing Memory Recall")
print("=" * 60)
print("🔄 Starting fresh - NO conversation history passed\n")

# Completely new conversation - Claude should remember from memory
conversation2 = chat_with_claude(
    "What do you remember about me and my interests?"
)

### 🔍 Understanding Check

**What just happened?**
1. You started a new conversation with no context
2. Claude used the memory tool to VIEW stored memories
3. It retrieved the information from the `.md` files on disk
4. It responded with personalized information about you

**This is different from regular chat history!**
- Regular chat: Context passed in every message (limited to session)
- Memory: Stored separately as files, accessible across any session

## Session 3: Personalized Recommendation

Let's ask Claude to use its memory to give us personalized recommendations.

In [ ]:
print("=" * 60)
print("SESSION 3: Personalized Recommendation")
print("=" * 60)
print("🎯 Testing if Claude uses memories for personalization\n")

conversation3 = chat_with_claude(
    "Based on what you know about me, recommend a Python library "
    "I might like for data visualization."
)

### 🎯 Checkpoint: Part 3 Complete!

You've successfully:
- ✅ Started new conversations without context
- ✅ Verified memory persistence across sessions
- ✅ Seen personalized responses based on memories

**Progress**: [███░░] 60% complete

🎊 **AMAZING!** Claude remembered across independent sessions!

---
# Part 4: Updating and Managing Memories ✏️

## What You'll Learn
How Claude dynamically updates memories as new information becomes available.

## Your Goal
Update existing memories with new preferences and verify the changes.

## Session 4: Update Your Preferences

Let's tell Claude about some changes in your preferences.

In [ ]:
print("=" * 60)
print("SESSION 4: Updating Memories")
print("=" * 60)
print("✏️ Telling Claude about preference changes\n")

conversation4 = chat_with_claude(
    "Update your memory about me: I've recently started using PyTorch "
    "for deep learning and I'm also interested in transformer models. "
    "Also, I now prefer using Jupyter Lab instead of just VSCode."
)

### 🔍 What to Look For

Claude may:
- Use `str_replace` to update existing memory
- OR create a new memory file
- OR append to existing content

Claude decides the best approach autonomously!

## Session 5: Verify the Update

Let's check if the memory was actually updated.

In [ ]:
print("=" * 60)
print("Verifying Memory Update")
print("=" * 60)

conversation5 = chat_with_claude(
    "What do you know about my deep learning interests?"
)

## View All Memory Files

Let's look at the complete state of our memory store.

In [ ]:
print("📊 Complete Memory Files State:")
print("=" * 60)

memory_files = list(MEMORY_DIR.glob('*.md'))
print(f"Total memory files: {len(memory_files)}")
print(f"Memory directory: {MEMORY_DIR.absolute()}")
print(f"File names: {[f.name for f in memory_files]}\n")

for md_file in sorted(memory_files):
    print(f"\n[{md_file.name}]")
    print(f"Created: {datetime.fromtimestamp(md_file.stat().st_ctime).isoformat()}")
    print(f"Size: {md_file.stat().st_size} bytes")
    print(f"\nFull Content:")
    print(md_file.read_text())
    print("-" * 60)

In [ ]:
print("📊 Memory Usage Analytics")
print("=" * 60)

memory_files = list(MEMORY_DIR.glob('*.md'))
total_memories = len(memory_files)
total_size = sum(f.stat().st_size for f in memory_files)

print(f"Total memory files: {total_memories}")
print(f"Total storage used: {total_size} bytes ({total_size/1024:.2f} KB)")
print(f"Memory directory: {MEMORY_DIR.absolute()}")
print(f"\nMemory file names: {[f.name for f in memory_files]}")

print("\n📅 Memory Timeline:")
for md_file in sorted(memory_files, key=lambda f: f.stat().st_ctime):
    created = datetime.fromtimestamp(md_file.stat().st_ctime).isoformat()[:19]
    size = md_file.stat().st_size
    print(f"  {created} - {md_file.name} ({size} bytes)")

print("\n" + "=" * 60)
print("🏆 CONGRATULATIONS! Tutorial Complete!")
print("=" * 60)
print(f"\n💡 Your memory files are saved in: {MEMORY_DIR.absolute()}")
print("💡 You can open them in any text editor to view or edit them!")
print("💡 They persist across kernel restarts and notebook sessions!")

### 🎯 Checkpoint: Part 5 Complete!

You've successfully:
- ✅ Generated personalized content
- ✅ Tested cross-session memory
- ✅ Analyzed memory usage
- ✅ Built a complete memory-enabled system

**Progress**: [█████] 100% complete

# 🏆 TUTORIAL COMPLETE!

You're now a Claude Memory Expert!

---
# 🎓 What You've Accomplished

## Skills Mastered

✅ **Memory System Architecture**
- Understand client vs. Claude roles
- Know how tool use patterns work
- Grasp memory persistence concepts

✅ **Implementation Skills**
- Set up AWS Bedrock with Claude Sonnet 4.5
- Implement memory storage layer
- Handle tool use requests
- Manage conversation loops

✅ **Practical Applications**
- Store information across sessions
- Retrieve memories contextually
- Update memories dynamically
- Build personalized interactions

## Real-World Use Cases

You can now build:
- 💬 Personalized chatbots that remember user preferences
- 🤝 AI assistants for long-term projects
- 📚 Learning systems that track progress
- 🎯 Context-aware recommendation engines
- 💼 Customer service bots with persistent context

---
# 🚀 Next Steps

## Immediate Next Steps

1. **Explore Your Memory Files**
   - Open the `memories/` directory in your file browser
   - Edit a `.md` file directly and see Claude use the updated content
   - Try version controlling them with git

2. **Test Persistence**
   - Restart the kernel and re-run from cell-1
   - Your memory files will still be there!
   - Claude will load them automatically

3. **Build a Simple App**
   - Create a command-line chatbot
   - Add it to a Discord bot
   - Integrate with a web application

## Advanced Explorations

**Production Memory Systems**
- Database integration (PostgreSQL, DynamoDB)
- Cloud storage (S3, GCS) for `.md` files
- Memory size limits and cleanup policies
- Multi-user support with user-specific directories

**Advanced Patterns**
- Semantic memory search with embeddings
- Automatic memory summarization
- Context-aware memory loading
- Memory conflict resolution

**Integration Projects**
- Flask/FastAPI web applications
- Discord/Slack bots
- Voice assistants
- Mobile app backends

## Learning Resources

📖 **Documentation**
- [Complete Setup Guide](docs/tutorials/claude-memory-quickstart.md)
- [AWS Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [Anthropic API Docs](https://docs.anthropic.com/)

💡 **Try These Challenges**
1. Build a bot that learns your daily schedule
2. Create a study assistant that tracks what you've learned
3. Make a recipe recommender that remembers your preferences
4. Build a project manager that remembers team members' skills

---
# 📚 Additional Resources

## Troubleshooting

**Common Issues:**
- Memory not persisting? Check that `memory_store` isn't being reset
- AWS errors? Verify credentials and model access
- Tool use not working? Confirm `anthropic_beta` flag is set

**Get Help:**
- See detailed troubleshooting: `docs/tutorials/claude-memory-quickstart.md#troubleshooting`
- Check AWS CloudWatch logs for detailed errors
- Review the verification script in Part 1

## Code Reference

Save this notebook as your reference! You can use it as a template for your own projects.

## Share Your Creation

Built something cool with Claude's memory? Share it with the community!

---

# 🎉 Congratulations!

You've completed the Claude Memory tutorial and are ready to build amazing memory-enabled AI applications!

**Remember**: Claude manages the intelligence of memory - you just provide the storage. This simple division of responsibility enables incredibly powerful applications.

Happy building! 🚀